### Учебный проект курса "Теория вероятностей и математическая статистика"
## Оценка возможности предсказания вероятности заключения сделки на предоставленных данных с достаточной надежностью 
**(по проектам малого производственного предприятия типа job-shop )**

In [560]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.options.display.max_columns = 100

In [561]:
data = pd.read_csv('job-projects.csv', sep=";")

In [562]:
data.head()

,project_id,in_work,Summ,work_summ,Sebest,o_pl,o_rent,Manager,client_projects,client_projects_calc,client_orders,client_konv,client_bill_middle,client_bill_middle_dc,client_pl_middle,client_projects_rent_middle,client_valid
0,12087,0,62135.88,45976.02,19393.98,26913.70,58.5,23.0,1,1,1,71.43,46388.94,33679.28,18724.43,54.92,1
1,11259,1,225455.78,120758.54,49676.97,74720.11,61.9,23.0,0,0,0,20.00,225455.78,120758.54,74720.11,61.90,1
2,12089,1,3700.00,3083.33,2228.09,855.24,27.7,22.0,0,0,0,91.43,50717.51,41792.53,21277.71,18.80,0
3,11803,1,4618.99,3849.16,1966.02,1883.14,48.9,22.0,1,0,0,91.43,50717.51,41792.53,21277.71,18.80,0
4,12150,1,16167.78,8478.55,4453.69,4310.27,50.8,23.0,74,70,64,85.71,16673.53,9724.47,4786.17,49.40,1


In [563]:
data.shape

(1150, 17)

## Разбиение на тестовый и валидационный сеты, подготовка данных

In [564]:
df = data.loc[:]
df.loc[df["in_work"]==0, "in_work"]=-1
df.describe()

,project_id,in_work,Summ,work_summ,Sebest,o_pl,o_rent,Manager,client_projects,client_projects_calc,client_orders,client_konv,client_bill_middle,client_bill_middle_dc,client_pl_middle,client_projects_rent_middle,client_valid
count,1150.000000,1150.000000,1.150000e+03,1.150000e+03,1.150000e+03,1.150000e+03,1150.000000,1148.000000,1150.000000,1150.000000,1150.000000,1150.000000,1.150000e+03,1.150000e+03,1150.000000,1150.000000,1150.000000
mean,12530.205217,0.027826,3.368178e+05,1.763592e+05,9.366011e+04,8.787821e+04,36.973739,23.362369,1.287826,1.066957,0.546087,50.649313,8.419611e+04,5.050333e+04,22841.526878,16.278965,0.840000
std,483.015668,1.000048,1.352400e+06,6.972112e+05,3.732077e+05,3.977392e+05,283.614561,1.353498,3.726943,3.402227,2.704645,37.877704,1.937552e+05,1.206731e+05,55986.440074,143.620433,0.366766
min,10869.000000,-1.000000,6.440400e+02,-1.062450e+04,1.040000e+03,-8.583453e+05,-8990.200000,11.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,-44768.890000,-1298.040000,0.000000
25%,12148.250000,-1.000000,2.011150e+04,1.296856e+04,5.379642e+03,6.435302e+03,38.025000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,1.000000
50%,12539.500000,1.000000,4.837146e+04,2.731151e+04,1.301145e+04,1.374849e+04,52.950000,23.000000,1.000000,0.000000,0.000000,57.140000,3.162631e+04,2.029085e+04,9750.190000,30.410000,1.000000
75%,12929.750000,1.000000,1.598046e+05,9.244716e+04,4.586260e+04,4.384014e+04,66.300000,24.000000,1.000000,1.000000,0.000000,85.710000,6.998990e+04,4.179253e+04,21277.710000,54.750000,1.000000
max,13376.000000,1.000000,1.774500e+07,9.497500e+06,5.736922e+06,6.762336e+06,359.000000,25.000000,74.000000,70.000000,64.000000,100.000000,1.575000e+06,1.312500e+06,506079.760000,93.030000,1.000000


In [565]:
#Не берем поле Менеджер - оно категориальное и требует дополнительной обработки

fields=["Summ", "work_summ", "Sebest", "o_pl", "o_rent", "client_projects", "client_projects_calc", "client_orders", "client_konv", "client_bill_middle", "client_pl_middle", "client_projects_rent_middle", "client_valid"]



Разобьем данные на обучающий и валидационный сеты

In [566]:
from sklearn.model_selection import train_test_split

train, valid, y_train, y_valid = train_test_split(df.loc[:, fields], df.loc[:, ["in_work"]], test_size=0.3, random_state=7)
train.shape, valid.shape,  y_train.shape, y_valid.shape

((805, 13), (345, 13), (805, 1), (345, 1))

In [567]:
train.head()

,Summ,work_summ,Sebest,o_pl,o_rent,client_projects,client_projects_calc,client_orders,client_konv,client_bill_middle,client_pl_middle,client_projects_rent_middle,client_valid
799,32553.2,27127.67,16938.10,10189.57,37.6,0,0,0,0.0,0.00,0.00,0.00,1
96,54725.3,32644.26,16261.10,17123.74,52.5,7,5,3,0.0,0.00,0.00,0.00,1
1114,729780.0,205615.00,113517.91,111265.42,54.1,0,0,0,0.0,0.00,0.00,0.00,1
946,99005.8,29269.83,9493.34,22311.49,76.2,0,0,0,0.0,0.00,0.00,0.00,1
503,994998.0,199782.01,34918.34,194834.29,97.5,1,1,1,37.5,371204.33,71694.62,82.43,1


Нормализуем значения параметров

In [568]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

def prepare_data(data, isTrain):
    if isTrain is True:
        scaled = scaler.fit_transform(data)
    else: 
        scaled = scaler.transform(data)
        
    ones = np.ones((scaled.shape[0], 1))
    x = np.hstack([ones, scaled])  
    return  x


In [569]:
x=prepare_data(train.loc[:, fields], True)
y=y_train.values.T[0]

x2=prepare_data(valid.loc[:, fields], False)
y2=y_valid.values.T[0]
x.shape, y.shape, x2.shape, y2.shape

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


((805, 14), (805,), (345, 14), (345,))

Построим модель логистической регрессии методом градиентного спуска.

In [570]:
#В качестве расстояния берем евклидово между векторами
from scipy.spatial.distance import euclidean

In [571]:
def sigmoid(x):
    """Сигмоида.
    """
    return 1 / (1 + np.exp(-x))

In [572]:
def partial_derivative(j, b, x, y):
    """Частная производная функционала Q по переменной b_j.
    """
    return -sum(
        x[i, j] * y[i] * (1 - sigmoid(b.dot(x[i]) * y[i]))
        for i in range(x.shape[0])
    )

In [573]:
def gradient(b, x, y):
    """Вектор градиента.
    """
   
    return np.array([
        partial_derivative(j, b, x, y)
        for j in range(b.shape[0])
    ])

In [574]:
def gradient_descent_step(lambda_, b, x, y):
    """Один шаг градиентного спуска.
    """
    minus=lambda_ * gradient(b, x, y)
    
    return b - minus

In [581]:
#В качестве стартового берем вектор с нулями
b_0 = np.zeros(x.shape[1])
b = b_0

for i in range(1, 10 ** 5):
    lambda_=0.01/i
    
    b_new = gradient_descent_step(lambda_, b, x, y)
    diff = euclidean(b, b_new)
    if diff < 0.001:
        print(f'success on step {i}\n')
        
        break
    b = b_new

b = b_new.reshape(-1, 1)
b

success on step 90



array([[-0.3900818 ],
       [-0.53387022],
       [-0.49039695],
       [-0.28148672],
       [-0.61341543],
       [-0.30067757],
       [-0.02913996],
       [ 0.03231162],
       [-0.02741305],
       [ 2.69923751],
       [ 0.21203123],
       [ 0.0660471 ],
       [ 0.04790598],
       [ 0.14083599]])

In [582]:
z = np.array([sigmoid(z_i) for z_i in x.dot(b)])
z2 = np.array([sigmoid(z_i) for z_i in x2.dot(b)])

In [583]:
z_train = np.array([
    1 if z_ > 0.5 else -1
    for z_ in z
])
z_valid = np.array([
    1 if z_ > 0.5 else -1
    for z_ in z2
])

In [584]:
from sklearn.metrics import accuracy_score

In [585]:
accuracy_score(y, z_train), accuracy_score(y2, z_valid)


(0.8472049689440994, 0.8521739130434782)

Модель выдает хорошие показатели как на учебном сете, так и для валидационных данных. 
Более подробный анализ прогностической модели на отдельных проектах показал, что для новых клиентов вероятность продажи проекта оценивается как низкая и уровень выставляемой цены большого влияния не оказывает - возможно, будет правильно сделать отдельные модели для новых клиентов и клиентов заказывавших ранее.